# ParFuMor (version YAML & Objets)

Attention les règles qui ne changent pas le radical donnent lieu à des mauvais découpages des formes...

In [15]:
# -*- coding: utf8 -*-
from os.path import expanduser
import itertools
import yaml,YamlDuplicates
from yaml.constructor import ConstructorError
import warnings
import ParFuMor as PFM
from ParFuMor import *
import pickle
from IPython.display import HTML, display
from cellbell import ding

In [16]:
home = expanduser("~")
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/"
annee="19"
numerosKalabas=[1,2,3,4,5]
numerosKalabas=[4]
nomKalabas=[repertoire+annee+"-K%d/"%num for num in numerosKalabas]

#nomKalabas=[home+"/Dropbox/Partage-Broadway/GAM/Caches/NosCaches/Kalabas/K2-Kalaba0304/"]

lexiqueTestPrep=u"dans"
lexiqueTestNoun=u"loups"
lexiqueTestHyper=u"villagEOIs"

In [17]:
PFM.duplicateErrors=[]
for serie in nomKalabas:
    print
    print serie

    with open(serie+"Gloses.yaml", 'r') as stream:
        gloses=yaml.load(stream)
        PFM.gloses=gloses
    with open(serie+"Stems.yaml", 'r') as stream:
        try:
            stems=yaml.load(stream)
            PFM.stems=stems
        except ConstructorError,msg:
            print msg
            continue
    with open(serie+"Blocks.yaml", 'r') as stream:
        blocks=yaml.load(stream)
        PFM.blocks=blocks
    with open(serie+"Phonology.yaml", 'r') as stream:
        phonology=yaml.load(stream)
        PFM.phonology=phonology
    with open(serie+"MorphoSyntax.yaml", 'r') as stream:
        morphosyntax=yaml.load(stream)
        PFM.morphosyntax=morphosyntax

        
    for cat in morphosyntax["Attributs"]:
        if sorted(morphosyntax["Attributs"][cat])!=sorted(gloses[cat].keys()):
            warnings.warn("\n"+serie+"\nLes attributs de %s ne sont pas cohérents\nMorphosyntax => %s\nGloses => %s"%(cat,", ".join(morphosyntax["Attributs"][cat]),", ".join(gloses[cat].keys())))
    
    regles=Regles()
    PFM.regles=regles
    for categorie in blocks:
        regles.addBlocs(categorie,blocks[categorie])


    paradigmes=Paradigmes()
    PFM.paradigmes=paradigmes

    hierarchieCF=HierarchieCF()
    PFM.hierarchieCF=hierarchieCF
    lexique=Lexique()
    PFM.lexique=lexique


    for cat in gloses:
    #    print cat
        attributes=[]
        if gloses[cat]:
            if set(gloses[cat].keys())==set(morphosyntax["Attributs"][cat]):
                features=morphosyntax["Attributs"][cat]
    #            print "inhérent",features
            else:
                features=gloses[cat].keys()
    #            print "contextuel",features
            for attribute in features:
                attributes.append(gloses[cat][attribute])
            nuplets=(itertools.product(*attributes))
            for nuplet in nuplets:
                proprietes=[cat]
                for element in range(len(nuplet)):
                    proprietes.append(u"%s=%s"%(features[element],nuplet[element]))
                paradigmes.addForme(cat,proprietes)

    analyserGloses(gloses)
    analyserStems(stems)
        
    with open(serie+"Hierarchie-S2.pkl", 'wb') as output:
       pickle.dump(hierarchieCF, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Lexique-S2.pkl", 'wb') as output:
       pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Regles-S2.pkl", 'wb') as output:
       pickle.dump(regles, output, pickle.HIGHEST_PROTOCOL)


    PFM.lexique.lexemes[lexiqueTestPrep]

    mot=lexiqueTestNoun
    classesNom=PFM.lexique.formeLexeme[mot][0].split('.')[1:]
    [classeElement for classeElement in classesNom if classeElement in gloses["NOM"]["Genre"]]

    PFM.lexique.formeLexeme[lexiqueTestHyper][0]

if PFM.duplicateErrors:
    print
    print "=======ERREURS========"
    print
    for ligne in PFM.duplicateErrors:
        print ligne



/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/19-K4/
ADJ : Genre Nombre Cas
NOM : Genre Nombre Cas
DET : Genre Nombre Cas
VER : Trans Temps Pers Genre
head stems
head stems,VER
head stems,VER,VI
head stems,VER,VD
head stems,VER,VT
head stems,ADJ
head stems,NOM
head stems,NOM,N1
head stems,NOM,N2
head stems,NOM,N3
head stems,NOM,N4


In [18]:
print PFM.lexique.formeLexeme[lexiqueTestHyper][0]
print PFM.lexique.formeLexeme[u"enfANT"][0]
print PFM.lexique.formeLexeme[u"crocs"][0]

villageoise.N4.A
enfant.hyper.N2.B
croc.N1.B


In [19]:
paradigmes.getSigmas(["DET"])

[u'DET, Genre=A, Nombre=Sg, Cas=Erg',
 u'DET, Genre=A, Nombre=Sg, Cas=Abs',
 u'DET, Genre=A, Nombre=Sg, Cas=Dat',
 u'DET, Genre=A, Nombre=Sg, Cas=Obl',
 u'DET, Genre=A, Nombre=Du, Cas=Erg',
 u'DET, Genre=A, Nombre=Du, Cas=Abs',
 u'DET, Genre=A, Nombre=Du, Cas=Dat',
 u'DET, Genre=A, Nombre=Du, Cas=Obl',
 u'DET, Genre=A, Nombre=Pl, Cas=Erg',
 u'DET, Genre=A, Nombre=Pl, Cas=Abs',
 u'DET, Genre=A, Nombre=Pl, Cas=Dat',
 u'DET, Genre=A, Nombre=Pl, Cas=Obl',
 u'DET, Genre=B, Nombre=Sg, Cas=Erg',
 u'DET, Genre=B, Nombre=Sg, Cas=Abs',
 u'DET, Genre=B, Nombre=Sg, Cas=Dat',
 u'DET, Genre=B, Nombre=Sg, Cas=Obl',
 u'DET, Genre=B, Nombre=Du, Cas=Erg',
 u'DET, Genre=B, Nombre=Du, Cas=Abs',
 u'DET, Genre=B, Nombre=Du, Cas=Dat',
 u'DET, Genre=B, Nombre=Du, Cas=Obl',
 u'DET, Genre=B, Nombre=Pl, Cas=Erg',
 u'DET, Genre=B, Nombre=Pl, Cas=Abs',
 u'DET, Genre=B, Nombre=Pl, Cas=Dat',
 u'DET, Genre=B, Nombre=Pl, Cas=Obl',
 u'DET, Genre=C, Nombre=Sg, Cas=Erg',
 u'DET, Genre=C, Nombre=Sg, Cas=Abs',
 u'DET, Genr

In [20]:
case="CF=N3"
m=re.match(ur"(.*=)(.*)","CF=N1|N2")
if m:
    altTrait=m.group(1)
    altValeurs=m.group(2).split("|")
    print altTrait
    altTraits=[altTrait+v for v in altValeurs]
    print altTraits
    if any(t == case for t in altTraits):
        print "applies"
    else:
        print "does not apply"

CF=
['CF=N1', 'CF=N2']
does not apply


In [21]:
regles.getRules("NOM",'CF=N3, Genre=C, Nombre=Du, Cas=Acc')

[('1A23o3i', 'CF=N3|N4, Nombre=Du')]

In [22]:
test=Lexeme("RAD","DET","croc").getParadigm(lignes=["Genre"],colonne=["Nombre"])
test
#test.index.sortlevel(0,ascending=False)

del Cas


Nombre,Du,Pl,Sg
Genre,,,
A,"me-RAD-b,fu-me-RAD-b,mi-me-RAD-b,Si-me-RAD-b","Si-no-RAD-b,no-RAD-b,mi-no-RAD-b,fu-no-RAD-b","Si-No-RAD-b,No-RAD-b,mi-No-RAD-b,fu-No-RAD-b"
B,"me-RAD-k,fu-me-RAD-k,mi-me-RAD-k,Si-me-RAD-k","Si-no-RAD-k,no-RAD-k,fu-no-RAD-k,mi-no-RAD-k","Si-No-RAD-k,No-RAD-k,fu-No-RAD-k,mi-No-RAD-k"
C,"me-RAD-b,fu-me-RAD-b,mi-me-RAD-b,Si-me-RAD-b","Si-no-RAD-b,no-RAD-b,mi-no-RAD-b,fu-no-RAD-b","Si-No-RAD-b,No-RAD-b,mi-No-RAD-b,fu-No-RAD-b"
D,"me-RAD-S,fu-me-RAD-S,mi-me-RAD-S,Si-me-RAD-S","Si-no-RAD-S,no-RAD-S,fu-no-RAD-S,mi-no-RAD-S","Si-No-RAD-S,No-RAD-S,fu-No-RAD-S,mi-No-RAD-S"


In [23]:
cat="N"
triLex={}
for element in PFM.lexique.lexemes:
#    print element
    elementElts=element.split(".")
    nom=elementElts[0]
    if len(elementElts)==3:
        if elementElts[1].startswith(cat):
            if not elementElts[2] in triLex:
                triLex[elementElts[2]]=set()
            triLex[elementElts[2]].add((PFM.lexique.lexemes[element].stem,nom))            
triLex

{u'A': {('Torel', u'infirmi\xe8re'),
  ('kaleb', u'Kaleb'),
  ('katiSa', u'Katisha'),
  ('koZoN', u'viande'),
  ('kuNol', u'villageoise'),
  ('kuleN', u'coup'),
  ('kurat', u'lit'),
  ('lajot', u'sort'),
  ('libaw', u'chat'),
  ('melis', u'balai'),
  ('nabil', u'Nabil'),
  ('nalus', u'blessure'),
  ('noTepe', u'th\xe9'),
  ('vosaZ', u'for\xeat'),
  ('wejeT', u'nuit'),
  ('wejin', u'caf\xe9')},
 u'B': {('Napel', u'fruit'),
  ('Totok', u'combattant'),
  ('beran', u'croc'),
  ('dewiN', u'table'),
  ('gozodi', u'loup'),
  ('gumiv', u'combat'),
  ('lenome', u'milieu'),
  ('murez', u'chasseur'),
  ('nenum', u'mort'),
  ('perik', u'autruche'),
  ('samil', u'souris'),
  ('siren', u'plaine'),
  ('tarak', u'col\xe8re'),
  ('tejan', u'main'),
  ('wiDer', u'arbre')},
 u'C': {('SawoT', u'coussin'),
  ('Taner', u'serpent'),
  ('Zamar', u'corps'),
  ('duTar', u'village'),
  ('geZij', u'poisson'),
  ('juron', u'coyote'),
  ('laNuN', u'cri'),
  ('lejen', u'maison'),
  ('lobow', u'chatte'),
  ('maira', 

In [24]:
for l in PFM.lexique.lexemes:
    print l,PFM.lexique.lexemes[l].stem

courageux.A2 TimuN
entrer.VI.V1 mabuk
trois.A1 doreNu
donner.VD.V2 kusiD
fille.N2.D kaZoNi
main.N3.B tejan
offrir.VD.V2 kaduw
enfant.N3.D tarili
coussin.N2.C SawoT
rejoindre.VT.V2 mugal
planter.VT.V1 tenam
blessure.N1.A nalus
couteau.N4.D niled
Violette.N3.D vjolet
vers woreZ
courir.VI.V2 laNar
envahir.VT.V2 nujak
ENTRE bot
effrayé.A2 vozur
jaune.A2 zimoN
de tuw
Nicole.N1.D nikol
sort.N2.A lajot
chef.N4.D naber
devant foru
visage.N4.C zamak
loup.N1.B gozodi
milieu.N2.B lenome
lueur.N2.C rulid
profond.A1 nidir
suivant.A1 Tinuj
voir.VT.V1 toNake
disparaître.VI.V1 voniS
acheter.VD.V1 baliz
furieux.A2 surezi
Nabil.N1.A nabil
aller.VI.V2 peger
chatte.N3.C lobow
croc.N1.B beran
DEM ba
blessé.A1 berin
combattant.N4.B Totok
enfant.hyper.N2.B kaZaNo
DEF la
chat.N2.A libaw
serpent.N4.C Taner
chambre.N2.D lebaS
thé.N4.A noTepe
coup.N1.A kuleN
poisson.N1.C geZij
recouvrir.VT.V2 litam
sur Sol
villageois.N3.C pujuS
viande.N2.A koZoN
jeter.VT.V2 bawaN
cuisine.N4.D vuris
bas.A2 tarane
protéger.VT.V2 t

In [25]:
ding()